In [10]:
# Contiene información técnica y de mercado de automóviles. El objetivo es predecir 
# el precio del automóvil en función de sus atributos


# Evaluar el desempeño del modelo en el conjunto de Test con métricas como precisión, recall, F1-score

# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Cargar el dataset
data = pd.read_csv('Dataset_Regresion_Lineal/Car details v3.csv')

# Limpieza de datos
columns_to_drop = ['name']  # Eliminar columnas irrelevantes
data = data.drop(columns=columns_to_drop)
data = data.dropna()  # Eliminar filas con valores nulos

# Asegurarse de que 'selling_price' sea numérico
data['selling_price'] = pd.to_numeric(data['selling_price'], errors='coerce')
data = data.dropna(subset=['selling_price'])  # Eliminar filas con precios nulos

# Convertir variables categóricas a variables numéricas con One Hot Encoding
data = pd.get_dummies(data, drop_first=True)

# Separar características y el target (Precio)
X = data.drop('selling_price', axis=1)
y = data['selling_price']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Selección de características
selector = SelectKBest(score_func=f_regression, k=10)  # Ajusta 'k' según sea necesario
X_selected = selector.fit_transform(X_scaled, y)

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Configuración de hiperparámetros para GridSearchCV
model = Ridge()  # Cambia a Lasso() si lo prefieres

# Definir el rango de hiperparámetros
param_grid = {
    'alpha': [0.1, 1, 10, 100],  # Solo para Ridge o Lasso
    'fit_intercept': [True, False]  # Ajustar el término de intercepción
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_model = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Evaluar el modelo usando métricas de regresión
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'\nMean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')

# Convertir predicciones y valores reales a categorías (ejemplo: bajo, medio, alto)
# Definir límites para categorizar
bins = [0, 200000, 500000, 1000000]  # Ajusta estos límites según tu dataset
labels = ['bajo', 'medio', 'alto']
y_train_class = pd.cut(y_train, bins=bins, labels=labels, right=False)
y_test_class = pd.cut(y_test, bins=bins, labels=labels, right=False)
y_pred_class = pd.cut(y_pred, bins=bins, labels=labels, right=False)

# Evaluar el modelo usando métricas de clasificación
print("\nMétricas de clasificación:")
print(classification_report(y_test_class, y_pred_class))

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
print("\nMatriz de confusión:")
print(conf_matrix)




Mean Squared Error (MSE): 211303023964.91312
R² Score: 0.6960906398651248

Métricas de clasificación:


TypeError: '<' not supported between instances of 'str' and 'float'